> # Dataloader

In [ ]:
import os
import cv2
import torch
import pandas as pd
from skimage import io
from torch.utils.data.dataset import Dataset
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self,**config):
        self.root_dir = "../input/cassava-leaf-disease-classification/test_images/"
        self.labels = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
        self.H = config['h']
        self.W = config['w']

    def __getitem__(self,idx):
        img_name = os.path.join(self.root_dir, self.labels.iloc[idx,0])
        img = io.imread(img_name)
        img = cv2.resize(img, (self.H, self.W)) 
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        img = img.transpose((2, 0, 1))
        img = torch.tensor(img,dtype=torch.float)
        label = self.labels.iloc[idx,1]
        label = torch.tensor(label,dtype=torch.long)
        sample = {'image':img,'label':label}
        return sample

    def __len__(self):
        return len(self.labels)

# Test

In [ ]:
from sklearn.metrics import classification_report
class Test():
    def __init__(self,**config):
        self.results = None
        self.csv_path = "../input/cassava-leaf-disease-classification/sample_submission.csv"
        self.dataloader = config['dataloader']
        self.model_path = config['model_path']
        self.out_path = 'submission.csv'
        
    def predict(self,):
        model = torch.load(self.model_path)
        results = []
        for i,batch in enumerate(self.dataloader):
            image = batch['image']
            pred = model(image.cuda())
            results+=torch.max(pred.data,1)[1].cpu().detach().numpy().tolist()
        self.results = results
        return results
        
    def out(self,):
        out = pd.read_csv(self.csv_path)
        out['label'] = self.results
        out.to_csv(self.out_path,index=False)
    

# Model

In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, **kwargs):
        super(AlexNet, self).__init__()
        self.kwargs = kwargs
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, self.kwargs["num_classes"]),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.contiguous().view(x.size(0), 256 * 7 * 7)
        x = self.classifier(x)
        return x

# Main

In [ ]:
testdataset = MyDataset(**model_config)
testdataloader = DataLoader(testdataset, batch_size=1,shuffle=False) 

# 加载模型
config = {"h": 256, "w": 256, num_classes: 5, "model_path": "./input/", 'dataloader':testdataloader}

model = AlexNet(**config)
model.load_state_dict(torch.load())

test = Test(**config)
test.predict()
test.out()

In [9]:
import pandas as pd
out = pd.read_csv('submission.csv')

In [10]:
out

,Unnamed: 0,image_id,label
0,0,2216849948.jpg,3
